# Recording the Gaze movement of the user

In [2]:
import cv2
import mediapipe as mp
import time
import csv

# Initialize mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Function to extract eye landmarks and draw gaze point
def draw_eye_gaze(frame, landmarks, img_w, img_h):
    # Left eye indices
    left_eye_landmarks = [33, 133]  # Two landmark points to approximate left eye center
    right_eye_landmarks = [362, 263]  # Two landmark points to approximate right eye center

    # Get left and right eye center
    left_eye_center = ((landmarks[left_eye_landmarks[0]].x + landmarks[left_eye_landmarks[1]].x) / 2 * img_w,
                       (landmarks[left_eye_landmarks[0]].y + landmarks[left_eye_landmarks[1]].y) / 2 * img_h)
    right_eye_center = ((landmarks[right_eye_landmarks[0]].x + landmarks[right_eye_landmarks[1]].x) / 2 * img_w,
                        (landmarks[right_eye_landmarks[0]].y + landmarks[right_eye_landmarks[1]].y) / 2 * img_h)

    # Draw circles around the center of the eyes in green
    cv2.circle(frame, (int(left_eye_center[0]), int(left_eye_center[1])), 3, (0, 255, 0), -1)
    cv2.circle(frame, (int(right_eye_center[0]), int(right_eye_center[1])), 3, (0, 255, 0), -1)

    return left_eye_center, right_eye_center

# Open the webcam
cap = cv2.VideoCapture(0)

# Open the CSV file for writing
csv_file = open("eye_gaze_coordinates_mediapipe.csv", mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["Timestamp", "Left Eye X", "Left Eye Y", "Right Eye X", "Right Eye Y"])

start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image color space from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image to get face landmarks
    result = face_mesh.process(rgb_frame)

    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            img_h, img_w, _ = frame.shape

            # Get eye gaze and draw
            left_eye_center, right_eye_center = draw_eye_gaze(frame, face_landmarks.landmark, img_w, img_h)

            # Save the eye coordinates with timestamp to CSV
            timestamp = time.time() - start_time
            csv_writer.writerow([timestamp, left_eye_center[0], left_eye_center[1], right_eye_center[0], right_eye_center[1]])


    # Display the frame with gaze tracking
    cv2.imshow("Gaze Tracking", frame)

    # Stop after 30 seconds or if the user presses 'q'
    if time.time() - start_time > 30 or cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

# Close the CSV file
csv_file.close()


# Data acquisition

In [18]:
import pandas as pd
import numpy as np
df_gz = pd.read_csv("eye_gaze_coordinates_mediapipe.csv")
df_gz

,Timestamp,Left Eye X,Left Eye Y,Right Eye X,Right Eye Y
0,0.585586,343.250065,294.268556,448.424110,314.254117
1,0.607169,345.759268,294.456353,450.998478,311.815181
2,0.628272,351.225681,294.830031,461.466599,315.296760
3,0.648829,344.396782,294.290171,457.295456,312.312627
4,0.701491,344.919434,293.321643,458.182640,317.203302
...,...,...,...,...,...
880,29.889083,347.746582,235.022907,412.978153,242.324638
881,29.921418,350.615444,236.017570,415.139141,245.740929
882,29.952391,350.811214,238.285940,416.623840,242.392459
883,29.983659,348.664436,236.229708,412.091789,240.818324


In [25]:
# recording the averaged gaze data
time_data = df_gz["Timestamp"]
# left eye x
lex_data = df_gz["Left Eye X"]
ley_data = df_gz["Left Eye Y"]
rex_data = df_gz["Right Eye X"]
rey_data = df_gz["Right Eye Y"]

time_init = 0.0
time_end = 30.0
counter = 0
# left eye stack
left_eye_x = []
left_eye_y = []
# right eye stack
right_eye_x = []
right_eye_y = []
# dictionary for storing time stamp and avarege left eye data
lex = {}
ley = {}
# deictionary for string time stamp and averge right eye data
rex = {}
rey = {}
while time_data[counter] < time_end:
    if time_data[counter] >= (time_init + 1.0):
        # printing time_init
        #print (time_init)
        # updating the average data dictionary
        lex_dict = {time_init:np.mean(np.array(left_eye_x))}
        ley_dict = {time_init:np.mean(np.array(left_eye_y))}
        rex_dict = {time_init:np.mean(np.array(right_eye_x))}
        rey_dict = {time_init:np.mean(np.array(right_eye_y))}
        # updating the dictionary with new data
        lex.update(lex_dict)
        ley.update(ley_dict)
        rex.update(rex_dict)
        rey.update(rey_dict)        
        # reset coordinates stack to store new data
        left_eye_x = []
        left_eye_y = []
        right_eye_x = []
        right_eye_y = []
        # incrementing tme_init by 0.1
        time_init += 1.0
    else:
        left_eye_x.append(lex_data[counter])
        left_eye_y.append(ley_data[counter])
        right_eye_x.append(rex_data[counter])
        right_eye_y.append(rey_data[counter])
        counter += 1

In [30]:
# creating the panadas dataframe from the dictionary of gaze movement
df_gaze = pd.DataFrame({
    "Time_stamp": lex.keys(),
    "Left_eye_x": lex.values(),
    "Left_eye_y": ley.values(),
    "Right_eye_x": rex.values(),
    "Right_eye_y": rey.values()
})

In [31]:
df_gaze

,Time_stamp,Left_eye_x,Left_eye_y,Right_eye_x,Right_eye_y
0,0.0,344.687706,293.461264,456.939157,314.738135
1,1.0,337.166922,259.237811,438.300166,272.821482
2,2.0,342.216389,255.043929,441.800015,267.845791
3,3.0,341.889778,252.558529,442.209673,265.899461
4,4.0,343.190010,254.635813,443.404956,267.648791
5,5.0,339.703777,257.186399,439.691282,270.426138
6,6.0,335.433262,224.210141,430.626080,233.444645
7,7.0,339.687638,233.344850,435.763248,243.037989
8,8.0,335.540066,201.732300,418.998411,211.789563
9,9.0,347.719941,198.780274,409.257130,206.834773


In [32]:
df_gaze.to_csv("averaged_coordinates_data.csv")